In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/engtasm/English-Assamese Training Data 2025.csv
/kaggle/input/eng-asm/transformers/default/1/adapter_model.safetensors
/kaggle/input/eng-asm/transformers/default/1/training_args.bin
/kaggle/input/eng-asm/transformers/default/1/adapter_config.json
/kaggle/input/eng-asm/transformers/default/1/README.md
/kaggle/input/eng-asm/transformers/default/1/tokenizer_config.json
/kaggle/input/eng-asm/transformers/default/1/special_tokens_map.json
/kaggle/input/eng-asm/transformers/default/1/sentencepiece.bpe.model
/kaggle/input/eng-asm/transformers/default/1/added_tokens.json
/kaggle/input/engtasmtest/en-as.txt


In [2]:
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sacrebleu
!pip install -U bitsandbytes
!pip install datasets pandas sacremoses fasttext sentence-transformers sentencepiece scikit-learn
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!pip install spacy indic-transliteration
!python -m spacy download en_core_web_sm 
!pip install indic-nlp-library 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, TaskType, get_peft_model
import evaluate 

2025-06-11 12:04:02.451001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749643442.673020      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749643442.735897      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
dataset = load_dataset(
    'csv', 
    data_files="/kaggle/input/engtasm/English-Assamese Training Data 2025.csv", 
    split="train"
)

subset = dataset.shuffle(seed=42).select(range(30000))

dataset_split = subset.train_test_split(test_size=0.1) 
train_dataset = dataset_split['train']
test_dataset = dataset_split['test']

print(f"Train examples: {len(train_dataset)}, Test examples: {len(test_dataset)}")  

Train examples: 27000, Test examples: 3000


In [8]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("as")

def normalize_assamese(batch):
    return {'as': [normalizer.normalize(x) for x in batch['as']], 'en': batch['en']}

train_dataset = train_dataset.map(normalize_assamese, batched=True)
test_dataset=test_dataset.map(normalize_assamese, batched=True)

def remove_duplicates(batch):
    seen = set()
    filt = []
    for en, as_ in zip(batch['en'], batch['as']):
        key = (en, as_)
        if key not in seen:
            seen.add(key)
            filt.append(True)
        else:
            filt.append(False)
    return {"keep": filt}

train_dataset = train_dataset.map(remove_duplicates, batched=True).filter(lambda x: x['keep'])
test_dataset = test_dataset.map(remove_duplicates, batched=True).filter(lambda x: x['keep'])

print(f"After removing duplicates: {len(train_dataset)}")
train_dataset[:5]

def ratio_filter(en, as_):
    ratio = len(en.split()) / max(1, len(as_.split()))
    return 0.5 <= ratio <= 2.0

train_dataset = train_dataset.filter(lambda x: ratio_filter(x['en'], x['as']))
test_dataset = test_dataset.filter(lambda x: ratio_filter(x['en'], x['as']))

import string

def preprocess_text(batch):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    
    # Process English text
    processed_en = [text.lower().translate(translator) for text in batch['en']]
    
    # Process Assamese text (assuming 'as' is the key for Assamese text)
    processed_as = [text.lower().translate(translator) for text in batch['as']]
    
    return {'en': processed_en, 'as': processed_as}

# Then apply text preprocessing
train_dataset = train_dataset.map(preprocess_text, batched=True)
test_dataset = test_dataset.map(preprocess_text, batched=True)

print(f"After preprocessing: {len(train_dataset)}")
train_dataset[:5]

import unicodedata

def to_halfwidth(text):
    return unicodedata.normalize('NFKC', text)

train_dataset = train_dataset.map(lambda x: {
    'en': to_halfwidth(x['en']),
    'as': to_halfwidth(x['as'])
})
test_dataset = test_dataset.map(lambda x: {
    'en': to_halfwidth(x['en']),
    'as': to_halfwidth(x['as'])
})
print(test_dataset[:3])
print(train_dataset[:3])

import fasttext

# Make sure lid.176.ftz is in /kaggle/working or input path
ft_model = fasttext.load_model("lid.176.ftz")

def lang_filter(text, lang):
    pred = ft_model.predict(text.replace(' ', '_'), k=1)
    code = pred[0][0].replace('__label__', '')
    return code == lang

train_dataset = train_dataset.filter(lambda x: lang_filter(x['en'], 'en') and lang_filter(x['as'], 'as'))
test_dataset = test_dataset.filter(lambda x: lang_filter(x['en'], 'en') and lang_filter(x['as'], 'as'))

print(f"After language filtering: {len(train_dataset)}")
train_dataset[:3]

train_dataset = train_dataset.remove_columns('keep')
test_dataset = test_dataset.remove_columns('keep')

def length_filter(en, as_):
    return len(en.split()) <= 150 and len(as_.split()) <= 150

train_dataset = train_dataset.filter(lambda x: length_filter(x['en'], x['as']))
test_dataset = test_dataset.filter(lambda x: length_filter(x['en'], x['as']))

print(f"After length filtering: {len(train_dataset)}")
train_dataset[:3] 

import string

translator = str.maketrans('', '', string.punctuation)

def remove_punctuation(batch):
    return {
        'en': [text.translate(translator) for text in batch['en']],
        'as': [text.translate(translator) for text in batch['as']],
    }

# Apply to both train and test datasets
train_dataset = train_dataset.map(remove_punctuation, batched=True)
test_dataset = test_dataset.map(remove_punctuation, batched=True)

def clean_whitespace(batch):
    return {
        'en': [x.strip().replace("  ", " ") for x in batch['en']],
        'as': [x.strip().replace("  ", " ") for x in batch['as']]
    }

train_dataset = train_dataset.map(clean_whitespace, batched=True)
test_dataset=test_dataset.map(clean_whitespace,batched=True)

import pandas as pd

data = pd.DataFrame(train_dataset[:10])  # Load first 10 rows
print(data['en'][3])
print(data['as'][3])

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/20829 [00:00<?, ? examples/s]

Map:   0%|          | 0/2356 [00:00<?, ? examples/s]

Map:   0%|          | 0/20829 [00:00<?, ? examples/s]

Map:   0%|          | 0/2356 [00:00<?, ? examples/s]

3 4  a what happens at a wedding feast in cana
ঈশ্বৰৰ ইচ্ছাক পূৰণ কৰিবলৈ দৃঢ় সংকল্প প্ৰকাশ কৰিলে


In [9]:
print("Train samples:")
for i in range(3):
    print(f"EN: {train_dataset[i]['en']}")
    print(f"AS: {train_dataset[i]['as']}")
    print()

Train samples:
EN: the prime minister called for discussions on matters such as judicious use of water resources better technology for storage and use of latest technology in farming during this krishi kumbh
AS: কৃষি কুম্ভ অনুষ্ঠানত জলসম্পদৰ উপযুক্ত ব্যৱহাৰ খাদ্যশস্য মজুত ৰখাৰ ক্ষেত্রত অত্যাধূনিক প্ৰযুক্তিৰ ব্যৱহাৰ আৰু কৃষিখণ্ডত আধুনিক প্ৰযুক্তিৰ ব্যৱহাৰৰ দৰে গুৰুত্বপূর্ণ বিষয়ক লৈ প্ৰধানমন্ত্রীয়ে আলোচনা কৰাৰ আহ্বান জনায় ।

EN: my full time service has enriched my life more than any secretarial job ever could have
AS: কিয়নো পূৰ্ণ সময় সেৱাৰ কাৰণে মই জীৱনত ইমানেই আনন্দিত আৰু উন্নত হোৱা অনুভৱ কৰিছোঁ যি মোক তেনে চাকৰিয়ে কেতিয়াও দিব নোৱাৰিলেহেঁতেন ।

EN: a brutal coarse or ignorant act
AS: নিষ্ঠুৰ কঠোৰ বা অবিবেচক কাৰ্য্য



In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set target language code (must match NLLB language codes)
tgt_lang = "asm_Beng"
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    forced_bos_token_id=forced_bos_token_id
)


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/94.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [11]:
def preprocess_examples(examples):
    # Set source and target languages
    tokenizer.src_lang = "eng_Latn"
    tokenizer.tgt_lang = "asm_Beng"

    # Tokenize inputs and targets
    model_inputs = tokenizer(
        examples["en"],
        max_length=256,
        padding="longest",
        truncation=True
    )
    
    labels = tokenizer(
        examples["as"],
        max_length=256,
        padding="longest",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply to datasets
train_dataset = train_dataset.map(preprocess_examples, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_examples, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/20829 [00:00<?, ? examples/s]

Map:   0%|          | 0/2356 [00:00<?, ? examples/s]

In [12]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  
    #use_dora=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
device = torch.device("cuda")

model=model.to(device)


trainable params: 56,623,104 || all params: 3,401,486,336 || trainable%: 1.6647


In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

In [14]:
bleu_metric = evaluate.load("sacrebleu")


In [15]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [transliterate(p, sanscript.DEVANAGARI, sanscript.BENGALI) for p in decoded_preds]
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=3e-5, 
    warmup_steps=1000,
    lr_scheduler_type="cosine", 
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
   # save_steps=1000,
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
    optim="adamw_torch_fused",
    adam_beta1=0.9,
    adam_beta2=0.98,
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

model.config.use_cache = False

# trainer.train()

/tmp/ipykernel_35/1960032874.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Bleu
1,5.433600,5.098196,14.356930


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


TrainOutput(global_step=1302, training_loss=6.822598365045363, metrics={'train_runtime': 7203.4135, 'train_samples_per_second': 2.892, 'train_steps_per_second': 0.181, 'total_flos': 2.2101375895732224e+16, 'train_loss': 6.822598365045363, 'epoch': 1.0})

In [26]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Bleu
1,4.965400,4.970937,15.406525


TrainOutput(global_step=1302, training_loss=4.97460459344398, metrics={'train_runtime': 6855.4225, 'train_samples_per_second': 3.038, 'train_steps_per_second': 0.19, 'total_flos': 2.2101375895732224e+16, 'train_loss': 4.97460459344398, 'epoch': 1.0})

In [30]:
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/sentencepiece.bpe.model',
 './final_model/added_tokens.json')

In [31]:
import shutil

# Create a ZIP file from the final_model folder
shutil.make_archive('/kaggle/working/final_model', 'zip', './final_model')


'/kaggle/working/final_model.zip'

In [35]:
import os
os.path.exists('/kaggle/working/final_model.zip')  # should return True


True

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
# trainer.save_model("./final_model")
# tokenizer.save_pretrained("./final_model")

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model and tokenizer from Kaggle input directory
model_path = "/kaggle/input/eng-asm/transformers/default/1"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    #quantization_config=quant_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# Set target language
tgt_lang = "asm_Beng"
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
model.config.forced_bos_token_id = forced_bos_token_id
model.config.use_cache = True

# Load Manipuri test sentences
test_file = "/kaggle/input/engtasmtest/en-as.txt"
with open(test_file, "r", encoding="utf-8") as f:
    manipuri_sentences = [line.split("\t")[0].strip() for line in f if line.strip()]

from tqdm import tqdm  

eng_predictions = []
for sentence in tqdm(manipuri_sentences, desc="Translating"):
    inputs = tokenizer(
        f"eng_Latn {sentence}",
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=256,
        num_beams=5
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    eng_predictions.append(decoded)

# Save predictions
with open("myresult.txt", "w", encoding="utf-8") as f:
    for pred in eng_predictions:
        f.write(pred + "\n")


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/94.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Translating:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Translating: 100%|██████████| 1000/1000 [34:44<00:00,  2.08s/it]
